### PULIZIA DATASET

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import re


In [8]:
df = pd.read_csv('mdl_bl_top_rated.csv', index_col=0)
df.head()

,title,year,country,rating,watchers,favorites,episodes,genres,synopsis,url
rank,,,,,,,,,,
1,ClaireBell,"Thai Drama - 2025, 8 episodes",NaN,8.9,NaN,NaN,NaN,NaN,NaN,https://mydramalist.com/777596-clairebell
2,Alice in Borderland,"Japanese Drama - 2020, 8 episodes",NaN,8.9,NaN,NaN,NaN,NaN,NaN,https://mydramalist.com/39537-imawa-no-kuni-no...
3,Me and Thee,"Thai Drama - 2025, 10 episodes",NaN,8.8,NaN,NaN,NaN,NaN,NaN,https://mydramalist.com/780120-me-and-thee
4,School Trip: Joined a Group I’m Not Close To,"Japanese Drama - 2025, 10 episodes",NaN,8.8,NaN,NaN,NaN,NaN,NaN,https://mydramalist.com/798660-shugakuryoko-de...
5,Jack & Joker: U Steal My Heart! (Uncut Ver.),"Thai Drama - 2024, 12 episodes",NaN,8.7,NaN,NaN,NaN,NaN,NaN,https://mydramalist.com/777409-jack-joker-u-st...


In [15]:
# 2. Definizione della funzione per estrarre i dati
# La colonna 'year' originale è sporca (es: "Thai Drama - 2025, 8 episodes")
# Questa funzione la smonta pezzo per pezzo.
def parse_info(row):
    text = str(row)
    # Dizionario vuoto per i dati che troveremo
    data = {
        'extracted_country': None, 
        'extracted_type': None, 
        'extracted_year': None, 
        'extracted_episodes': None
    }
    
    # Dividiamo la stringa in due parti usando " - " come separatore
    parts = text.split(' - ')
    if len(parts) >= 2:
        # PARTE SINISTRA: es. "Thai Drama"
        left_part = parts[0].strip()
        
        # Prendiamo la prima parola come Paese e il resto come Tipo
        if ' ' in left_part:
            data['extracted_country'] = left_part.split(' ')[0]
            data['extracted_type'] = ' '.join(left_part.split(' ')[1:])
        else:
            data['extracted_country'] = left_part
            
        # PARTE DESTRA: es. "2025, 8 episodes"
        right_part = parts[1].strip()
        
        # Cerchiamo l'anno (4 cifre consecutive) con le Espressioni Regolari (Regex)
        year_match = re.search(r'(\d{4})', right_part)
        if year_match:
            data['extracted_year'] = int(year_match.group(1))
            
        # Cerchiamo il numero di episodi (cifre seguite dalla parola 'episode')
        ep_match = re.search(r'(\d+)\s+episode', right_part)
        if ep_match:
            data['extracted_episodes'] = int(ep_match.group(1))
            
    return pd.Series(data)

In [16]:
# 3. Applicazione della funzione a tutto il dataset
# Questo crea un nuovo dataframe temporaneo con le colonne estratte
extracted_data = df['year'].apply(parse_info)

In [17]:
# 4. Unione e Pulizia finale
# Uniamo le nuove colonne al dataframe originale
df_clean = pd.concat([df, extracted_data], axis=1)

In [19]:
# Rimuoviamo le colonne originali che erano vuote o non servono più
cols_to_drop = ['country', 'watchers', 'favorites', 'episodes', 'genres', 'synopsis']
# filtra solo le colonne presenti per evitare KeyError
existing_cols = [c for c in cols_to_drop if c in df_clean.columns]
if existing_cols:
    df_clean = df_clean.drop(columns=existing_cols)

# Rinominiamo le nuove colonne per renderle più leggibili
df_clean = df_clean.rename(columns={
    'extracted_country': 'country',
    'extracted_type': 'type', 
    'extracted_year': 'year_released',
    'extracted_episodes': 'episodes'
})

In [21]:
# 5. Visualizzazione del risultato
print("Prime 5 righe del dataset pulito:")
cols_to_show = ['title', 'country', 'year_released', 'episodes', 'rating']
existing_cols_to_show = [c for c in cols_to_show if c in df_clean.columns]
print(df_clean[existing_cols_to_show].head())
if 'rating' not in existing_cols_to_show:
	print("Nota: la colonna 'rating' non è presente nel dataset e quindi è stata omessa.")

Prime 5 righe del dataset pulito:
                                             title   country  year_released  \
rank                                                                          
1                                       ClaireBell      Thai           2025   
2                              Alice in Borderland  Japanese           2020   
3                                      Me and Thee      Thai           2025   
4     School Trip: Joined a Group I’m Not Close To  Japanese           2025   
5     Jack & Joker: U Steal My Heart! (Uncut Ver.)      Thai           2024   

      episodes  
rank            
1            8  
2            8  
3           10  
4           10  
5           12  
Nota: la colonna 'rating' non è presente nel dataset e quindi è stata omessa.


In [45]:
# --- LA SOLUZIONE DEFINITIVA ---

# 1. Sovrascriviamo la colonna 'rank' con numeri da 1 alla fine
# Non la impostiamo come indice, la lasciamo come colonna normale.
df_clean['rank'] = range(1, len(df_clean) + 1)

# 2. Mettiamo 'rank' come prima colonna nell'ordine
cols_order = ['rank', 'title', 'country', 'type', 'year_released', 'episodes', 'rating', 'url']
# (Filtro di sicurezza per prendere solo le colonne che esistono)
cols_order = [c for c in cols_order if c in df_clean.columns]
df_clean = df_clean[cols_order]

# 3. SALVIAMO CON index=False
# Questo è il comando cruciale. Dice a Pandas:
# "Scrivi nel file SOLO le colonne che ho deciso io (rank, title...).
# NON scrivere i tuoi numeri di riga automatici a sinistra."
df_clean.to_csv('mdl_bl_top_rated_finale.csv', index=False)

print("File creato: 'mdl_bl_top_rated_finale.csv'")
print("Apri QUESTO file in Excel/Notepad per vedere che è corretto.")

File creato: 'mdl_bl_top_rated_finale.csv'
Apri QUESTO file in Excel/Notepad per vedere che è corretto.


In [46]:
df = pd.read_csv('mdl_bl_top_rated_finale.csv')
df.head()

,rank,title,country,type,year_released,episodes
0,1,ClaireBell,Thai,Drama,2025,8
1,2,Alice in Borderland,Japanese,Drama,2020,8
2,3,Me and Thee,Thai,Drama,2025,10
3,4,School Trip: Joined a Group I’m Not Close To,Japanese,Drama,2025,10
4,5,Jack & Joker: U Steal My Heart! (Uncut Ver.),Thai,Drama,2024,12
